In [2]:
import numpy as np
import math
import Bio

In [21]:
score_matrix = np.array([[-math.inf, -1, -1, -1, -1],
                         [-1, 1, -1, -1, -1],
                         [-1, -1, 1, -1, -1],
                         [-1, -1, -1, 1, -1],
                         [-1, -1, -1, -1, 1]])

def smtx(a,b):
    if a == "-":
        if b == "-":
            return -math.inf
        return -1
    if a == b:
        return 1
    else:
        return -1

testo = "GCATGCA"
testt = "GATTACA"

NWA now seems to work, just need to implement backtracking algorithmically to find optimal alignment/solution

In [30]:
def NWA(seqa,seqb):
    grid = np.zeros((len(seqa)+1,len(seqb)+1))
    srow = []
    for i in range(len(seqa)+1):
        srow.append(0-i)
    for i in range(len(seqb)+1):
        grid[i][0] = 0-i
    grid[0] = srow
    midgrid = grid[1:,1:]
    for x in range(1,len(seqa)):
        for y in range(1,len(seqb)):
            midgrid[x][y] = max((grid[x-1][y-1]+smtx(seqa[x],seqb[y])),
                             (grid[x-1][y]+smtx(seqa[x],"-")),
                             (grid[x][y-1]+smtx("-",seqb[y])))
    print(grid)

    return None

In [31]:
NWA(testo,testt)

[[ 0. -1. -2. -3. -4. -5. -6. -7.]
 [-1.  0.  0.  0.  0.  0.  0.  0.]
 [-2.  0. -1. -1. -1. -1. -1. -1.]
 [-3.  0.  0. -1. -1.  1. -1.  1.]
 [-4.  0. -1.  1.  0. -2. -2.  0.]
 [-5.  0. -1. -1. -1.  0.  0.  0.]
 [-6.  0. -1. -1.  0.  0.  1. -1.]
 [-7.  0. -1. -1. -2.  0. -1.  1.]]
